In [ ]:
from google.colab import drive 
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import csv 
from sklearn.model_selection import train_test_split
import matplotlib.ticker as ticker
import seaborn as sns

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/USYD/2021 S1/INFO4001/TabulaMuris_Heart_10X-RawCount-modifed.csv',header=None)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,16,17,18,19,20,22,24,25,26,27,28,29,30,31,32,34,35,37,38,39,40,41,42,43,44,46,47,48,49,50,51,52,54,55,56,57,58,59,61,62,64,65,66,67,68,69,70,71,72,73,74,75,76,77,79,80,81,82,83,85,86,87,88,89,90,91,92,94,95,96,97,98,99,101,102,103,104,105,106,107,108,109,111,112,114,115,116,117,118,119,120,121,122,123,125,126,128,129,131,132,133,134,135,136,137,138,139,140,141,142,145,146,147,149,151,152,153,154,155,156,157,158,159,160,161,162,164,165,166,167,168,169,170,171,172,173,175,177,179,180,181,182,183,184,186,187,188,191,195,199,200,201,202,204,205,206,215,216,223,224,227,230,234,238,243,245,246,248,249,250,251,262,275,276,284,285,287,290,293,302,303,304,306,313,314,315,316,317,319,328,329,335,337,338,339,340,341,347,349,350,351,354,357,360,361,362,367,368,369,370,371,372,373,377,379,380,381,383,384,385,387,389,390,391,392,393,395,401,402,403,404,405,406,407,408

In [ ]:
def shuffle_data(x):
    
    # save header
    tmp_head = x[0]

    #remove header 
    x = x[1:]

    # shuffle by row 
    np.random.shuffle(x)

    #stack header
    x = np.vstack((tmp_head,x))

    # shuffle by column after T transformation 
    x = x.T 

    tmp_head = x[0]
    x = x[1:]

    np.random.shuffle(x)

    x = np.vstack((tmp_head,x))

    return x 

In [ ]:
def remove_invalid_val(mt):
    inval_lst= ['nan','alpha.contaminated', 'beta.contaminated', 
    'delta.contaminated', 'Excluded', 'gamma.contaminated', 'miss', 'NA',
    'not applicable', 'unclassified', 'unknown', 'Unknown', 'zothers']
    print("before shape {}".format(mt.shape))
    lst = mt[0].astype(str)
    d_lst=[]

    for i in range(len(lst)):
        for j in range(len(inval_lst)):

            if lst[i] == inval_lst[j]:
                d_lst.append(i)
    print(len(d_lst))
    mt = np.delete(mt,d_lst,axis=1)
    print("after shape {}".format(mt.shape))
    return mt

In [ ]:
# data preprocessing and data split 

mt = df.to_numpy()
#mt = mt.T
mt = remove_invalid_val(mt)

#shuffle dataset by rows and cols
mt = shuffle_data(mt)

# get type for each sample 
mt_Y = mt.T[0][1:]

mt1 = mt[:,1:]
mt_X = mt1[1:]
mt_X = mt_X.astype('float64')


unique_classes = np.unique(mt_Y)

# split data to train and test 
x_train, x_test, y_train, y_test = train_test_split(
    mt_X, mt_Y, test_size=0.2,stratify=mt_Y, random_state=42)

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(unique_classes)
print(len(unique_classes))

before shape (22841, 7116)
2581
after shape (22841, 4535)
(3627, 22840) (907, 22840) (3627,) (907,)
['cardiac muscle cell' 'endocardial cell' 'endothelial cell'
 'epicardial adipocyte' 'erythrocyte' 'fibroblast' 'hematopoietic cell'
 'leukocyte' 'smooth muscle cell']
9


In [ ]:
#Different normalisation method: raw, CPM, minmax(CPM)

#cpm only
tmp_train = np.nan_to_num(x_train)
tmp_test = np.nan_to_num(x_test)
sum_train = np.sum(tmp_train,axis=1)
sum_train = sum_train.reshape(sum_train.shape[0],-1)
sum_test = np.sum(tmp_test,axis=1)
sum_test = sum_test.reshape(sum_test.shape[0],-1)
#print(tmp_train.shape)
#print(np.sum(tmp_train,axis=1))

#cpm_x_train = np.divide(tmp_train, np.sum(tmp_train,axis=1))



#print(a2.shape)
#cpm_x_train = tmp_train /a2
cpm_x_train = np.divide(tmp_train, sum_train)
cpm_x_test =  np.divide(tmp_test, sum_test)
print(cpm_x_train.shape)



#cpm with minmax
mnx = MinMaxScaler()

mnx_x_train = mnx.fit_transform(cpm_x_train)

mnx_x_test = mnx.transform(cpm_x_test)



(3627, 22840)


In [ ]:

label = unique_classes.tolist()


n_lst=[]
for i in range(len(y_train)):
  for j in range(len(label)):
    if y_train[i] == label[j]:

      n_lst.append(j)

y_new_train = np.array(n_lst,dtype='float64')

n_lst=[]
for i in range(len(y_test)):
  for j in range(len(label)):
    if y_test[i] == label[j]:
      n_lst.append(j)

y_new_test = np.array(n_lst,dtype='float64')

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(x_train,y_new_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [ ]:
y_pred = neigh.predict(x_test)



In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

out_acc = accuracy_score(y_new_test, y_pred)
print("Raw data Accuracy: {}".format(round(out_acc,4)))

out_f1 = f1_score(y_new_test, y_pred,average ='micro')
print("Raw data F1 Score(micro): {}".format(round(out_f1,4)))

out_f1 = f1_score(y_new_test, y_pred,average ='macro')
print("Raw data F1 Score(macro): {}".format(round(out_f1,4)))

out_f1 = f1_score(y_new_test, y_pred,average ='weighted')
print("Raw data F1 Score(weighted): {}".format(round(out_f1,4)))

Raw data Accuracy: 0.903
Raw data F1 Score(micro): 0.903
Raw data F1 Score(macro): 0.6621
Raw data F1 Score(weighted): 0.9023


In [ ]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(cpm_x_train,y_new_train)
y_pred = neigh.predict(cpm_x_test)

out_acc = accuracy_score(y_new_test, y_pred)
print("CPM data Accuracy: {}".format(round(out_acc,4)))

out_f1 = f1_score(y_new_test, y_pred,average ='micro')
print("CPM data  F1 Score(micro): {}".format(round(out_f1,4)))

out_f1 = f1_score(y_new_test, y_pred,average ='macro')
print("CPM data  F1 Score(macro): {}".format(round(out_f1,4)))

out_f1 = f1_score(y_new_test, y_pred,average ='weighted')
print("CPM data  F1 Score(weighted): {}".format(round(out_f1,4)))


CPM data Accuracy: 0.9405
CPM data  F1 Score(micro): 0.9405
CPM data  F1 Score(macro): 0.8222
CPM data  F1 Score(weighted): 0.9378


In [ ]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(mnx_x_train,y_new_train)
y_pred = neigh.predict(mnx_x_test)

out_acc = accuracy_score(y_new_test, y_pred)
print("mixmax(CPM) data Accuracy: {}".format(round(out_acc,4)))

out_f1 = f1_score(y_new_test, y_pred,average ='micro')
print("mixmax(CPM) data  F1 Score(micro): {}".format(round(out_f1,4)))

out_f1 = f1_score(y_new_test, y_pred,average ='macro')
print("mixmax(CPM) data  F1 Score(macro): {}".format(round(out_f1,4)))

out_f1 = f1_score(y_new_test, y_pred,average ='weighted')
print("mixmax(CPM) data  F1 Score(weighted): {}".format(round(out_f1,4)))

mixmax(CPM) data Accuracy: 0.602
mixmax(CPM) data  F1 Score(micro): 0.602
mixmax(CPM) data  F1 Score(macro): 0.3739
mixmax(CPM) data  F1 Score(weighted): 0.5947
